In [1]:
# this is a little trick to make sure the the notebook takes up most of the screen:
from IPython.display import HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')  


<p><b><font size="200"> A Small Showcase of CREST </font></b></p>
Thank you to [Jupyter](http://jupyter.org/) and [Binder](https://mybinder.org/) for building the base of this interactive CREST demo.

This is a preliminary implementation. It is meant as a showcase and is not yet production ready.  
The source code can be found here: https://github.com/stklik/CREST/

## How to use this Jupyter notebook:  
Select a cell (with code) and click the 'Run' button above (or pressing `Ctrl+Enter`) to execute it.
If there is output, it will be shown directly underneath the cell. 
Remember that executing a cell sends the cell's code to an interactive Python interpreter and the memory state is carried forward.
This means that you the execution order is important. Also, repeatedly executing a cell will not replace the previous state, but add to it.
This means that if you e.g. define a cell with the code `a = a + 1` (where `a` is a variable defined in a previous cell) then each execution of the cell will increment the value of `a` by one.

**This notebook** was designed to execute all cells from top to bottom. You can either iteratively execute each cell or simply execute all at once:  
Menu `Cell` -> `Run all`  

# First import the necessary CREST model libraries

In [ ]:
# import all model concepts
from crestdsl.model import *

## Define the resources that we use in the system

In [ ]:
# define the resources in a class, so they are bundled. This is syntactic sugar.
# it is important however that none of the resources has the same name as any of the ports!

# currently supported value domains: REAL, INTEGER and lists of discrete values
class Resources(object):
    electricity = Resource("Watt", REAL)
    switch = Resource("switch", ["on", "off"])
    light = Resource("Lumen", INTEGER)
    counter = Resource("Count", INTEGER)
    time = Resource("minutes", REAL)
    celsius = Resource("Celsius", REAL)
    fahrenheit = Resource("Fahrenheit", REAL)
    

## Let's define an entity
Entities derive from the basic 'Entity' class.  
They can define:
 - inputs, outputs and local variables
 - states and the current state
 - transitions between states
 - influences between ports (incl. transformations)
 - updates related to a port, which continuously evaluate while in a certain state
 - actions, which are updates that are executed when a transition is fired (semantically with a dt of 0, here we don't even use the parameter)

In [ ]:
class LightElement(Entity):
    """This is a definition of a new Entity type. It derives from CREST's Entity base class."""
    
    """we define ports - each has a resource and an initial value"""
    electricity = Input(resource=Resources.electricity, value=0)
    light = Output(resource=Resources.light, value=0)
    
    """automaton states - don't forget to specify one as the current state"""
    on = State()
    off = current = State()
    
    """transitions and guards (as lambdas)"""
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.electricity.value >= 100))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.electricity.value < 100))
    
    """
    update functions. They are related to a state, define the port to be updated and return the port's new value
    Remember that updates need two parameters: self and dt.
    """
    @update(state=on, target=light)
    def set_light_on(self, dt=0):
        return 800

    @update(state=off, target=light)
    def set_light_off(self, dt=0):
        return 0

## Plotting
We use the *elk* module to create an interactive system representation 
(based on the [`mxGraph`](https://jgraph.github.io/mxgraph/) diagram library and the [`elkjs`](https://github.com/OpenKieler/elkjs) layout engine.)
The layout and production of the HTML version takes a few seconds 

This interactive diagramming is continuously extended. The currently supported features are:

- Move objects around if the automatic layout does not provide an sufficient result.
- Select ports and states to see their outgoing arcs (blue) and incoming arcs (red).
- Hover over transitions, influences and actions to display their name and short summary.
- Double click on transitions, influences and actions you will see their source code.
- There is a *hot corner* on the top left of each entity. You can double-click it to collapse the entity. This feature is useful for CREST diagrams with many entities. *Unfortunately a software issue prevents the expand/collapse icon not to be displayed. It still works though (notice your cursor changing to a pointer)*

 
**GO AHEAD AND TRY IT**

In [ ]:
# import the plotting libraries that can visualise the CREST systems
from crestdsl.ui import elk

elk.plot(LightElement())

### Alternative plotting
There exists a fast, static plotter based on `graphviz` and `dot`.
You can use the `dotter` module to plot a quick picture. BUT you *cannot* modify it or influence the output.

In [ ]:
# import the dotter
from crestdsl.ui import dotter

dotter.plot(LightElement())

## Define another entity (without transitions)
Note that this one does only have one state and no transitions.

In [ ]:
class HeatElement(Entity):
    """ Ports """
    electricity = Input(resource=Resources.electricity, value=0)
    switch = Input(resource=Resources.switch, value="off")  # the heatelement has its own switch
    heat = Output(resource=Resources.celsius, value=0)      # and produces a celsius value (i.e. the temperature increase underneath the lamp)
    
    """ Automaton (States) """
    state = current = State() # the only state of this entity
    
    """Update"""
    @update(state=state, target=heat)
    def heat_output(self, dt):
        # When the lamp is on, then we convert electricity to temperature at a rate of 100Watt = 1Celsius
        if self.switch.value == "on":
            return self.electricity.value / 100
        else:
            return 0

# show us what it looks like
elk.plot(HeatElement())

# An Adder Entity

CREST does not specify a special connector type that defines what is happening for multiple incoming influence, etc. Instead standard entities are used to define add, minimum and maximum calculation which is then written to the actual target port using an influence.

We call such entities *logical*, since they don't have a real-world counterpart.

In [ ]:
# a logical entity (this one sums two values)
class Adder(LogicalEntity):
    heat_in = Input(resource=Resources.celsius, value=0)
    room_temp_in = Input(resource=Resources.celsius, value=22)
    temperature = Output(resource=Resources.celsius, value=22)
    
    state = current = State()
    @update(state=state, target=temperature)
    def add(self, dt):
        return self.heat_in.value + self.room_temp_in.value
    
elk.plot(Adder())  # try adding the display option 'show_update_ports=True' and see what happens!

# A more complex entity (with subentities)
The GrowLamp is a standard entity just like the ones above, except...  
... it defines subentities !!

In [ ]:
class GrowLamp(Entity):
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    electricity = Input(resource=Resources.electricity, value=0)
    switch = Input(resource=Resources.switch, value="off")
    heat_switch = Input(resource=Resources.switch, value="on")
    room_temperature = Input(resource=Resources.fahrenheit, value=71.6)
    
    light = Output(resource=Resources.light, value=3.1415*1000) # note that these are bogus values for now
    temperature = Output(resource=Resources.celsius, value=4242424242) # yes, nonsense..., they are updated when simulated
    
    on_time = Local(resource=Resources.time, value=0)
    on_count = Local(resource=Resources.counter, value=0)
    
    """ - - - - - - - SUBENTITIES - - - - - - - - - - """
    lightelement = LightElement()
    heatelement = HeatElement()
    adder = Adder()
    
    
    """ - - - - - - - INFLUENCES - - - - - - - - - - """
    """
    Influences specify a source port and a target port. 
    They are always executed, independent of the automaton's state.
    Since they are called directly with the source-port's value, a self-parameter is not necessary.
    """
    @influence(source=room_temperature, target=adder.room_temp_in)
    def celsius_to_fahrenheit(value):
        return (value - 32) * 5 / 9
    
    # we can also define updates and influences with lambda functions... 
    heat_to_add = Influence(source=heatelement.heat, target=adder.heat_in, function=(lambda val: val))
    
    # if the lambda function doesn't do anything (like the one above) we can omit it entirely...
    add_to_temp           = Influence(source=adder.temperature, target=temperature)
    light_to_light        = Influence(source=lightelement.light, target=light)
    heat_switch_influence = Influence(source=heat_switch, target=heatelement.switch)
    
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    on = State()
    off = current = State()
    error = State()
    
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch.value == "on" and self.electricity.value >= 100))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch.value == "off" or self.electricity.value < 100))
    
    # transition to error state if the lamp ran for more than 1000.5 time units
    @transition(source=on, target=error)
    def to_error(self):
        """More complex transitions can be defined as a function. We can use variables and calculations"""
        timeout = self.on_time.value >= 1000.5
        heat_is_on = self.heatelement.switch.value == "on"
        return timeout and heat_is_on
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    # LAMP is OFF or ERROR
    @update(state=[off, error], target=lightelement.electricity)
    def update_light_elec_off(self, dt):
        # no electricity
        return 0

    @update(state=[off, error], target=heatelement.electricity)
    def update_heat_elec_off(self, dt):
        # no electricity
        return 0
    
    
    
    # LAMP is ON
    @update(state=on, target=lightelement.electricity)
    def update_light_elec_on(self, dt):
        # the lightelement gets the first 100Watt
        return 100
    
    @update(state=on, target=heatelement.electricity)
    def update_heat_elec_on(self, dt):
        # the heatelement gets the rest
        return self.electricity.value - 100
        
    @update(state=on, target=on_time)
    def update_time(self, dt):
        # also update the on_time so we know whether we overheat
        return self.on_time.value + dt
        
    """ - - - - - - - ACTIONS - - - - - - - - - - """
    # let's add an action that counts the number of times we switch to state "on"
    @action(transition=off_to_on, target=on_count)
    def count_switching_on(self):
        """
        Actions are functions that are executed when the related transition is fired.
        Note that actions do not have a dt.
        """
        return self.on_count.value + 1

# create an instance!
elk.plot(GrowLamp())